# Euclidean Distance Degree

A real algebraic variety is the common zero set of polynomials $f_1, \ldots, f_m \in \mathbb{R}[x_1,\ldots,x_n]$ denoted by $V(f_1,\ldots,f_m)$.

Consider the distance from a point $u \in \mathbb{R}^n$ to the variety $X=V(f_1,\ldots,f_m)$. What is the nearest point on $X$ to $u$ with respect to the euclidean distance? How many critical points are there? If we count the number of critical points over the complex numbers then this number will almost always be the same. It is called the *Euclidean distance degree* of $X$. 

<figure>
<img src="images/ed.png" alt="ED" style="width: 500px;"/>
<figcaption style="color:DIMGRAY">The real critical points of the euclidean distance function for the point $u= [-0.32, -0.1]$
    and $X=V((x^4 + y^4 - 1)(x^2 + y^2 - 2) + x^5y)$</figcaption>
</figure>

 In the following we want so show **you** can solve this problem on your own using techniques from numerical algebraic geometry.

For a given $u \in \mathbb{R}^n$ and $X=V(f_1,\ldots, f_m)$ we want to solve the problem:

$$\min =||x-u||_2=:d_u(x) \quad \text{subject to} \quad x \in X$$

Considering the geometry of the problem  you can see that a point $x^*$ is a critical point of $d_u$ if and only if
$x^* - u$ is orthogonal to the tangent space of $X$ at $x^*$, or formally
        \begin{equation}(x^* - u) \perp T_{(x^*)}X \,.\end{equation}

**Exercise**: Write down definining equations for our problem.

<details>
 <summary>A solution:</summary>
    Assume X=$V(f_1,\ldots, f_m)$ and $\dim(X)=n-m$. Let $J(x)$ the Jacobian of $f=(f_1,\ldots, f_m)$ where the $i$-th row consists of the partial derivatives of $f_i$.
    Then, critical points satisfy the equations
    $$x-u =J(x)^T \lambda$$
        $$f(x) = 0$$
            $$\lambda \in \mathbb{R}^m$$
    Note: These are the same equations you get from applying Lagrange multipliers to the optimization problem. Why?
 </details>

Now that we have the defining equations for our problem let's solve it using [HomotopyContinuation.jl](https://www.juliahomotopycontinuation.org).

In [ ]:
# load HomotopyContinuation.jl
using HomotopyContinuation

To make our life easier we are already provided with some helper functions by the `TAGSS` package.

In [ ]:
# load the TAGSS helper package
using TAGSS

We start by definining a variety. Let's take the variety from the beginning:
$$V((x^4 + y^4 - 1)(x^2 + y^2 - 2) + x^5y)$$

In [ ]:
# We can define variables like this
@polyvar x y

Now let's set $f$ to the polynomial $(x^4 + y^4 - 1)(x^2 + y^2 - 2) + x^5y$

In [ ]:
# TODO: Define f

Let's visualize $V(f)$ to see whether we got it right. 

In [ ]:
visualize(f, show_axis=true, scene_resolution=(500,500), x_min=-2, x_max=2)

Now we need to define the critical equations:

In [ ]:
# we define new variables u₁, u₂ and λ₁
@polyvar u[1:2] λ[1:1]
# define the jacobian of F
J = differentiate([f], [x,y])
# J' defines the transpose of J
# TODO: Set F to be the critical equations
F = 

The TAGSS package can also generate equations for you with
```julia
F, u = eq_equations(f)
```

Now that we have our equations let's define the point $u_0=[-0.32, -0.1]$

In [ ]:
# TODO: Define the point u₀
u₀ = 

Our equation $F$ is still parameterized by $u$. So we have to make the substitution $u \Rightarrow u_0$.

In [ ]:
F_u₀ = [subs(f, u => u₀) for f in F]

$F_{u_0}$ is now a polynomial system of 3 equations in 3 unknowns. Homotopy continuation methods allow us to compute *all* solutions of this polynomial system. These methods first compute all solution over the *complex numbers*. Here we use the `solve` routine from HomotopyContinuation.jl as a black-box. You can find more background information on this method in the [How does it work?](https://www.juliahomotopycontinuation.org/guides/how-does-it-work/) guide on the [HomotopyContinuation.jl homepage](https://www.juliahomotopycontinuation.org).

In [ ]:
res = solve(F_u₀)

We find that our problem has **36** solutions over the complex numbers (which is also the generic number of solutions).
Thus the *Euclidean distance degree* is 36.
Furthermore we can see that there are **8** real solutions.

Now let's extract the real points.

In [ ]:
# Let's get all real solutions
real_sols = real_solutions(res)
# We have to remove our artifical variable λ₁
ed_points = map(p -> p[1:2], real_sols)

Now let's find the optimal solution:

In [ ]:
using LinearAlgebra

dist, idx = findmin([norm(x - u) for x in ed_points])
println("Optimal solution: ", ed_points[idx], " with distance ", sqrt(dist), " to u₀")

We can also visualize our solutions:

In [ ]:
visualize_ed(f, u₀, ed_points; show_axis=true, scene_resolution=(600,600))

Great! This looks like our picture from the beginning.

## Surfaces in $\mathbb{R}^3$

The TAGSS package also provides us with the possibility to visualize surfaces in $\mathbb{R}^3$.

In [ ]:
@polyvar x y z
g = (0.3*x^2+0.5z-0.3x+1.2*y^2-1.1)^2+(0.7*(y+0.5x)^2+y+1.2*z^2-1)^2-0.3

Let's visualize the surface:

In [ ]:
visualize(g; wireframe=true, scene_resolution=(600,600)) #you can also set wireframe=false to see a colored version of the surface

Let's pick a point in $\mathbb{R}^3$

In [ ]:
u₀ = [0.5, 2.5, -1.2]

Now `TAGSS` also provides the black box routine `ed` to compute all real critical points. So let us be a little bit lazy and just use this function.

In [ ]:
ed_pts = ed(g, u₀)

And let's visualize them:

In [ ]:
visualize_ed(g, u₀, ed_pts; show_axis=false, scene_resolution=(800,800))

You should also be able to get a rotating 3D picture from

In [ ]:
display(visualize_ed(g, u₀, ed_pts; show_axis=false, scene_resolution=(800,800)));

## Exercises

1) Compute the ED Degree for your favourite curve or surface and visualize it.

2) If $X=V(f_1,\ldots,f_m)$ is not a complete intersection, i.e., $\dim(X) > n - m$ the formulation
 $$\begin{align}x-u &=J(x)^T \lambda  \\ f(x) &= 0 \\ \lambda &\in \mathbb{C}^m \end{align}$$
    doesn't describe a zero dimensional solution set any more. Why? How can you fix this?